In [2]:
import datetime
import urllib
prefix = '/Users/dd/Downloads/.metagraph_run33/'

fServer = open(f'{prefix}/server.log')
lines = fServer.readlines()

downloaded_sras = set()
build_sras = set()
clean_sras = set()
transfer_sras = set()
ndownloaded_sras = set()
nbuild_sras = set()
nclean_sras = set()
ntransfer_sras = set()
time_first = 0
time_last = 0
for l in lines:
    if not 'ack' in l:
        continue
    split_l = l.split(' ')
    date_time = datetime.datetime.strptime(split_l[0] + ' ' + split_l[1], '%Y-%m-%d %H:%M:%S,%f')
    parsed = urllib.parse.parse_qs(split_l[4])
    sra_id = parsed['id'][0]
    if time_first == 0:
        time_first = date_time
    time_last = date_time
    if '/ack/download' in l:
        downloaded_sras.add(sra_id)
    elif '/ack/build' in l:
        build_sras.add(sra_id)
    elif '/ack/clean' in l:
        clean_sras.add(sra_id)
    elif '/ack/transfer' in l:
        transfer_sras.add(sra_id)
    elif '/nack/download' in l:
        ndownloaded_sras.add(sra_id)
    elif '/nack/build' in l:
        nbuild_sras.add(sra_id)
    elif '/nack/clean' in l:
        nclean_sras.add(sra_id)
    elif '/nack/transfer' in l:
        ntransfer_sras.add(sra_id)
    
print(f'Downloaded {len(downloaded_sras)}, Built {len(build_sras)}, Cleaned {len(clean_sras)}, Transferred {len(transfer_sras)}')
print(f'Not downloaded {len(ndownloaded_sras)}, Not built {len(nbuild_sras)}, Not cleaned {len(nclean_sras)}, Not transferred {len(ntransfer_sras)}')
    


Downloaded 71917, Built 71514, Cleaned 71506, Transferred 71506
Not downloaded 2795, Not built 398, Not cleaned 0, Not transferred 0


In [3]:
from collections import defaultdict
import urllib.parse

download_size = 0
download_size_hist = defaultdict(int)
download_sizes = []
download_time = 0
sra_to_size = {}
ndownload_size_hist = defaultdict(int)
coverage = []
coverage_size = []
coverage_total_size = 0

for l in lines:
    if not 'ack/down' in l:
        continue
    parsed = urllib.parse.parse_qs(l.split(' ')[4])
    sra_id = parsed['id'][0]
    size = float(parsed['size_mb'][0])
    
    if '/ack/download' in l:
        kmer_coverage = float(parsed['kmer_coverage'][0])
        unique_kmers = float(parsed['kmer_count_unique'][0])
        coverage.append(int(kmer_coverage))
        coverage_size.append(size)
        coverage_total_size += size
        download_size_hist[int(size/100) if int(size/100)<200 else 200] += 1
        download_sizes.append(size/1e3)
        sra_to_size[sra_id] = size
        
        if sra_id in transfer_sras:
            download_time += int(l.split('&')[2].split('=')[1])
            download_size = download_size + size
    elif '/nack/download' in l:
        ndownload_size_hist[int(size/100) if int(size/100)<200 else 200] += 1
print(f'Total download size (of finished SRAs): {round(download_size,2)}MB')
print(f'Total download time (of finished SRAs): {download_time}s')
print(f'Download bandwidth (of finished SRAs): {round(download_size/download_time,2)}MB/s/machine')


Total download size (of finished SRAs): 42968024.5MB
Total download time (of finished SRAs): 8884976s
Download bandwidth (of finished SRAs): 4.84MB/s/machine


In [4]:
time = 0
size_download = 0
size_build = 0
build_size_hist = defaultdict(int)
build_sizes = []
build_size_to_time = {}
nbuild_size_hist = defaultdict(int)
nbuild_sizes = []

for d in lines:
    if not 'ack/build' in d:
        continue
    if '/ack/build' in d:
        sra_id = d.split('&')[1].split('=')[1]
        sz = sra_to_size[sra_id] #float(d.split('&')[4].split('=')[1])
        build_size_hist[int(sz/100) if int(sz/100)<200 else 200] += 1
        build_sizes.append(sz/1e3)
        tm = float(d.split('&')[2].split('=')[1])
        size_build += float(d.split('&')[4].split('=')[1])
        build_size_to_time[sz] = tm
        if sra_id in transfer_sras:
            time = time + tm
            size_download += sz
    elif '/nack/build' in d:
        sz = sra_to_size[sra_id]
        nbuild_size_hist[int(sz/100) if int(sz/100)<200 else 200] += 1
        nbuild_sizes.append(sz/1e3)

print(f'Total build time (of finished SRAs): {time}s')
print(f'Total build size (of finished SRAs): {size_build}MB')
print(f'Build bandwidth is {size_download/time}MB/s')


Total build time (of finished SRAs): 5976542.0s
Total build size (of finished SRAs): 14948430.130000051MB
Build bandwidth is 7.1894457530793305MB/s


In [5]:
time = 0
clean_size = 0
for d in lines:
    if not 'ack/clean' in d:
        continue
    sra_id = d.split('&')[1].split('=')[1]
    if sra_id in transfer_sras:
        time = time + float(d.split('&')[2].split('=')[1])
        clean_size = clean_size + float(d.split('&')[3].split('=')[1])
        
    
print(f'Total clean time (of finished SRAs): {time}s')
print (f'Clean bandwidth is {download_size} {time} {round(download_size/time,2)}MB/s')
print(f'Compression factor is {download_size/clean_size}')
print(f'Overall processing bandwidth is {(time_last-time_first).total_seconds()/download_size}')
print(time_last, time_first)

Total clean time (of finished SRAs): 13436753.0s
Clean bandwidth is 42968024.5000002 13436753.0 3.2MB/s
Compression factor is 42.3576248639823
Overall processing bandwidth is 0.0017624621536882538
2020-04-16 18:51:48.289000 2020-04-15 21:49:38.772000


In [ ]:
# import matplotlib.pyplot as plt

# plt.bar(list(download_size_hist.keys()), download_size_hist.values(), color='g')
# plt.ylabel('count')
# plt.xlabel('size (100s of MB)')
# plt.title('SRA distribution by size (download successful)')
# plt.show()

# plt.bar(list(ndownload_size_hist.keys()), ndownload_size_hist.values(), color='g')
# plt.ylabel('count')
# plt.xlabel('size (100s of MB)')
# plt.title('SRA distribution by size (download failed)')
# plt.show()

# plt.hist(download_sizes, bins=range(0,80))

In [ ]:
import collections

# plt.figure(1)
# plt.bar(list(build_size_hist.keys()), build_size_hist.values(), color='g')
# plt.ylabel('count')
# plt.xlabel('size (100s of MB)')
# plt.title('SRA distribution by size (build successful)')
# #plt.show()

# plt.figure(2)
# plt.bar(list(nbuild_size_hist.keys()), nbuild_size_hist.values(), color='g')
# plt.ylabel('count')
# plt.xlabel('size (100s of MB)')
# plt.title('SRA distribution by size (build failed)')
# #plt.show()

# plt.figure(3)
# plt.title('SRA histogram')
# plt.hist(build_sizes, bins=[0, 10, 20, 30, 40, 50])
# plt.hist(nbuild_sizes, bins=[0, 10, 20, 30, 40, 50])
# #plt.show()

# plt.figure(4)
# plt.title('Build time by size')
# od = collections.OrderedDict(sorted(build_size_to_time.items()))
# # plt.plot(list(od.keys()), list(od.values()))




In [ ]:
# plt.figure(1)
# plt.title('K-mer coverage')
# plt.hist(coverage, bins=range(0,40), weights=coverage_size, density=True)
# plt.show()